# Projeto Urna - EDA dos dados das eleições presidenciais de 2022

## Sobre o projeto:

As eleições presidenciais de 2022 foram algumas das mais intensas que ocorrem desde a redemocratização (tanto o segundo turno quanto o primeiro turno), e assim que os resultados do segundo turno saíram, me senti praticamente intimado a fazer uma análise dos dados das eleições, de ambos os turnos. A princípio, eu quis fazer **apenas** das eleições presidenciais, mas optei no final por fazer para todos os cargos.

O dataset das eleições foi obtido no site do [TSE](https://dadosabertos.tse.jus.br/dataset/resultados-2022), vindo juntamente com um arquivo .pdf, que é o dicionário de dados para entendermos o que cada atributo representa. O arquivo original é em formato .csv e a codificação do mesmo é "Latin-1" (mas por via de regra, passei como parâmetro "ISO 8859-1", que é o "nome" oficial da codificação). Eu poderia trabalharcom os dados das eleições envolvendo o estado do Rio de Janeiro inteiro, mas nesta fase vou me ater a cidade onde moro (Nova Friburgo, região serrana do RJ).

Como não há microdados (um microdado, segundo a definição do [CETIC](https://www.cetic.br/microdados/), é a menor fração de um dado coletado em pesquisa - ou seja, representa uma pessoa ou uma resposta individual), o número de votos está agrupado no dataset, hierarquicamente como "estado - cidade - zona eleitoral - seção eleitoral".

Todos os arquivos necessários para a empreitada estarão presentes no mesmo diretório, então zero preocupações quanto a isso. Minha escolha para converter tudo em _.xlsx_ ao fim por ser o formato padrão do Excel (caso haja alguém com vontade de manipular diretamente a planilha.

## Informação importante

Os datasets das eleições estão separados. Em um dataset estão os resultados das eleições para deputados federais e estaduais, senadores e governadores. Em outro, encontram-se os dados das eleições presidenciais (não entendi muito bem o porquê da separação, mas acredito que a divisão seja critério do TSE). Portanto, o dataset das eleições "normais" será chamado apenas de _"df_votacao"_ e o dataset das eleições presidenciais será chamado de _'df_votacao_presidente'_. No site do TSE, o dataset presidencial encontra-se com o título _'Presidente - Votação por seção eleitoral - 2022'_ e o dataset 'normal' se encontra com o título _'RJ - Votação por seção eleitoral - 2022'_.

Analisando o tamanho dos arquivos - eleição presidencial e eleição normal -, dá pra imaginar o porquê do TSE ter optado por essa divisão. Se fóssemos ter em apenas um arquivo todos os dados das eleições, primeiro e segundo turno para **todos** os cargos, haveria um volume MUITO grande de dados. E também, se pararmos pra pensar de forma lógica, vemos muito mais notícias ligadas a presidência do que da Câmara dos Deputados ou do Senado. Para os analistas de dados que produzem conteúdo para imprensa, é muito mais vantagem ter essa divisão. O trabalho fica mais focado onde deve ficar focado - _apenas estou dando os meus dois centavos sobre o assunto_.

## Importando as bibliotecas

Um desafio que coloquei para mim mesmo: Fazer o projeto utilizando DUAS bibliotecas de manipulação de dados. Uma delas é a mais que famosa Pandas e a outra, nem tão conhecida porém ganhando espaço, Polars.
Neste espaço, serão usadas APENAS as bibliotecas [Pandas](https://pandas.pydata.org) e [Numpy](https://numpy.org), importadas abaixo:

In [11]:
# Bibliotecas que utilizarei por aqui

import pandas as pd
import numpy as np
import os
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor
import openpyxl

# Para evitar repetições desnecessárias, escrevi uma função que descreve exatamente o que quero: 
# Que me mostre a quantidade de entradas nulas por coluna e descreva o tipo de dado em cada coluna, além
# do formato do dataframe (quantidade de linhas e colunas).

def avaliador(df):
    formato = df.shape
    print(f"Tamanho do DataFrame selecionado: {formato[1]} colunas e {formato[0]} linhas.\n")
    print("#" * 20)
    print("\n")

    # Exibindo os tipos de cada coluna
    print("\nTIPOS DAS COLUNAS:")
    for coluna in df.columns:
        print(f"Tipo da coluna '{coluna}': {df[coluna].dtype}")
    print("\n")
    print("#" * 20, "\n")

    # Quantidade de linhas nulas por coluna
    print("Quantidade de dados nulos por colunas do DataFrame selecionado: ")
    print(df.isnull().sum())
    print("\n")
    

## Criando o DataFrame com Pandas

Aqui é onde importo os dados, no formato .csv, utilizando o método "read_csv" e passando como parâmetros o "encoding" para cada  arquivo (ISO 8859-1 ou "Latin-1"). Também foi necessário determinar o "delimiter" em ";" - preferência pessoal).

**Informação importante**: Em cada célula, haverá um marcador - %%time - que irá informar quanto tempo levou para que cada operação seja realizada. É um dado importante, já que haverão comparações entre o tempo de execução do Pandas e do Polars.

In [12]:
%%time

# DataFrame das eleições normais
df_votacao = pd.read_csv('votacao_RJ_2022.csv', encoding='ISO-8859-1', delimiter=';')

#DataFrame das eleições presidenciais
df__votacao_presidente = pd.read_csv('votacao_BR_2022.csv', encoding='ISO-8859-1', delimiter=';')

CPU times: total: 53.9 s
Wall time: 54.2 s


### Podemos concluir que:
- Ambos os Dataframes posssuem 26 colunas - um número muito grande de colunas, sendo que não iremos precisar disso tudo
- O número de linhas nos dois não será contabilizado, já que a natureza de coleta dos dados é diferente em ambos os datasets
- Porém possuem a mesma estrutura (em nomes de colunas, natureza das informações e tipos primitivos) - o que será muito bom para a estratégia que vem a seguir
- Não há valores nulos nas linhas e colunas - são dois DataFrames bem cuidados, aparentemente
- Os tipos das colunas se dividem em "int64" e 'object' ('object' é a forma que o Pandas interpreta o tipo "str"). Talvez eu altere o tipo de dados para que o gasto de memória em cada operação fique 'redondinho', bem otimizado. No caso, é necessário? Como as operações então funcionando de forma fluida, eu diria que é mais para que o costume do "tuning" seja fixado.

## E qual vai ser a estratégia a partir de agora?

A partir daqui, vamos realizar a fusão dos DataFrames, depois iremos salvá-los em um arquivo .xlsx. Depois dessa etapa, iremos re-invocar o novo arquivo para que possamos trabalhar em cima dele mais tranquilamente.

## E por que fazer isso?

Por alguns motivos.
- O primeiro deles é por preguiça. E a preguiça, se bem aplicada, é um dos pilares da programação. Poupo a mim de escrever comandos redundantes e ainda por cima centralizo todos os dados em um só DataFrame. Mão na roda demais.
- Os DataFrames possuem a mesma estrutura, sendo um pra eleição presidencial (nos dois turnos) e o outro pra todo o restante dos cargos - uma informação que acho pontual: No RJ, a eleição para governador se deu em apenas um turno. Houve o mesmo resultado em mais 14 estados.
- Caso eu decida utilizar o dataset, já transformado do jeito que quero, como database em algum software de dataviz (Power BI, Qlik, Tableau, etc), não irei precisar utilizar várias bases de dados. Apenas uma já está de bom tamanho. Ajuda no consumo de memória.

## Então vamos lá: Realizando a fusão dos DataFrames

In [13]:
%%time
# Realizando a concatenação dos DataFrames. Vamos lá aos argumentos:
# axis = 0 - quero unir os DataFrames verticalmente, de acordo com as linhas
# ignore_index=True - quero resetar o índice do novo DataFrame, evitando algumas 'doideras'

df_eleicoes = pd.concat([df_votacao, df__votacao_presidente], axis=0, ignore_index=True)

CPU times: total: 891 ms
Wall time: 899 ms


## Analisando a saúde do novo DataFrame:

- Mesmo número de colunas de antes, agora com o objeto tendo um pouco mais de 12 milhões de linhas
- Mais uma vez, sem tipos nulos
- Os tipos variam entre "int64" e "object"

As colunas que ficarão no DataFrame serão:
- 'NR_TURNO' (representa o turno da eleição)
- 'NM_MUNICIPIO' - (nome do município, coluna mais importante por enquanto)
- 'NR_ZONA' - (Zona eleitoral)
- 'NR_SECAO' - (Seção eleitoral)
- 'CD_CARGO - (Cargo do político)
- 'NR_VOTAVEL' - (Número do político)
- 'QT_VOTOS' - (Quantidade de votos que o político recebeu naquela zona/seção eleitoral)
- 'NM_VOTAVEL' - (Nome do político)
- 'NM_LOCAL_VOTACAO' - (Endereço do local da votação) ¹
- 'DS_LOCAL_VOTACAO_ENDERECO' - (Nome do local da votação) ¹


**Nota**: Percebe-se que há uma redução IMENSA a ser feita; Os outros dados não são tão relevantes pra essa EDA. Não estou tirando a importância dos dados que irei excluir; se eu quisesse fazer um modelo de machine learning, eu manteria o campo 'DT_ELEICAO', por exemplo. São ideias mais a frente.

**Nota ¹**: As duas últimas colunas serão necessárias apenas temporariamente. Abaixo explico melhor.

## Refatoração e reestruturação do DataFrame

Como próxima etapa, irei primeiramente renomear as colunas do DataFrame principal para que eu possa ter uma padronização - parte de uma estratégia pra evitar ao máximo fazer coisas 'a mão'. Com as colunas renomeadas, irei selecionar apenas as informações pertinentes a Nova Friburgo - cidade a ser analisada. Por fim, vou fazer o casting das colunas numéricas do meu DataFrame principal - de int64 para int32.

(**Meus dois centavos sobre o assunto**: Uma coisa que eu gostaria que fosse possível é o casting de colunas do tipo 'object' para 'str' com limitação de caracteres. No SQL mesmo, é possível modificar o tipo de uma coluna usando, no MySQL mesmo, *'MODIFY coluna VARCHAR(número)'*. Uma solução mais elegante para o casting. Nem tudo nessa vida é perfeito, infelizmente.)

In [14]:
# A lista com todas as colunas que preciso usar:
colunas_a_manter = ['NR_TURNO',
                    'NM_MUNICIPIO',
                    'NR_ZONA',
                    'NR_SECAO',
                    'CD_CARGO',
                    'NR_VOTAVEL',
                    'QT_VOTOS',
                    'NM_VOTAVEL',
                    'NM_LOCAL_VOTACAO',
                    'DS_LOCAL_VOTACAO_ENDERECO']

# O novo DataFrame, com a lista sendo passada como argumento
df_eleicoes = df_eleicoes[colunas_a_manter]
df_eleicoes = df_eleicoes[df_eleicoes['NM_MUNICIPIO'] == 'NOVA FRIBURGO']

In [15]:
%%time
# Lista com as colunas que servirão como orientação para a ordenação de valores
ordem = ['NR_ZONA','NR_SECAO']

# Aqui é feita a ordenação do dataset
df_eleicoes = df_eleicoes.sort_values(by=ordem,axis=0, ascending=True, kind='quicksort')

CPU times: total: 15.6 ms
Wall time: 20.9 ms


## Conversão dos tipos

Aqui foi feita a conversão das colunas numéricas. De *'int 64'* para *'int32'*, foram testados vários algoritmos e as suas respesctivas velocidades. No final, com a ajuda do [José Paulo Costa](https://www.linkedin.com/in/josepaulocosta), o algoritmo escolhido foi implementado - utilizando uma máscara lógica e o método '.iloc'.

In [16]:
%%time
# Algoritmo de conversão otimizado:

# Passo (1)
mascara = (df_eleicoes.dtypes == 'int64').to_numpy()

# Passo (2)
df_eleicoes.iloc[:, mascara] = df_eleicoes.iloc[:, mascara].astype('int32', copy=False)

CPU times: total: 15.6 ms
Wall time: 9.97 ms


## Salvando o dataframe

Vou precisar salvar o dataframe para poder extrair alguns dados dele, em outro notebook (bagunça demais fazer tudo em apenas um notebook). Será salvo em formato .xlsx, por comodidade. 

In [17]:
df_eleicoes.to_excel('dataset_auxiliar.xlsx')

## Última etapa: Adição de alguns dados

Irei precisar de duas tabelas-dimensão: Uma tabela de bairros e uma tabela de partidos políticos. Isso irá facilitar a análise violentamente. O código delas foi feito em outro notebook, por razões de organização. Neste notebook só iremos usar as tabelas já prontas.

**E como essa facilitação vai rolar?** Fundindo as informações das duas tabelas dimensão com a tabela fato faz com que os dados fiquem centralizados em apenas uma tabela, evitando comandos de agregação *à posteriori* e, com isso, deixando o notebook mais "limpo". Essa é basicamente a última parte do *data munging*, posterior a isso faremos a análise de dados propriamente dita. 

In [18]:
# Importando a tabela de bairros
lista_locais = pd.read_excel('lista_locais.xlsx')

# Importando a tabela de partidos
lista_partidos = pd.read_excel('lista_partidos.xlsx')

In [19]:
# Primeira fusão feita: Dataset principal com a lista de locais de votação
df_eleicoes = df_eleicoes.merge(lista_locais, on='NM_LOCAL_VOTACAO', how='left')

In [20]:
# Aqui tive que fazer um 'desvio': Obtive uma nova coluna a partir dos dois primeiros dígitos dos números dos políticos
# Nova coluna obtida, fiz a conversão dela em 'int32' - já que ela foi convertida em str para a primeira manipulação
df_eleicoes['NUMERO'] = df_eleicoes.NR_VOTAVEL.astype(str).str[:2]
df_eleicoes['NUMERO'] = df_eleicoes['NUMERO'].astype('int32')

# Realizando a segunda fusão: O dataframe, já fundido com a tabela de locais, com a tabela de partidos
df_eleicoes = df_eleicoes.merge(lista_partidos, on='NUMERO', how='left')

# Essa fusão gera uma 'poluição' na forma de colunas 'Unnamed'. Invoco uma lista com todas essas colunas, primeiramente
colunas_irrelevantes = [cols for cols in df_eleicoes.columns if "Unnamed" in cols]

# Depois, é só deletar essas colunas usando o método '.drop'
df_eleicoes = df_eleicoes.drop(columns= colunas_irrelevantes)

In [21]:
# Agora é eliminar as colunas originais que não serão usadas
# Lista com todos os nomes das colunas
colunas_irrelevantes = ['NM_MUNICIPIO', 
                        'NM_VOTAVEL', 
                        'NM_LOCAL_VOTACAO',
                        'DS_LOCAL_VOTACAO_ENDERECO',
                        'ZONA',
                        'SECAO',
                        'ENDERECO',
                        'NUMERO',
                        'PARTIDO'
                       ]

# O 'drop' das colunas é feito
df_eleicoes = df_eleicoes.drop(columns=colunas_irrelevantes)

# Por último, vamos renomear todas as colunas para que seja mais fácil na hora de fazer a EDA
novos_nomes = ['TURNO','ZONA','SECAO','CARGO','NUMERO','VOTOS','BAIRRO','SIGLA','ALINHAMENTO']

# Atribuindo os novos títulos das colunas ao dataframe
df_eleicoes.columns = novos_nomes

## Adicionando mais algumas informações no dataframe

Precisamos de mais uma coluna, que explicite se o voto ao candidato foi válido ou não
E por que fazer isso? A urna eletrônica não contabiliza se o voto em candidaturas sub júdice (indeferidas a princípio mas que entraram com recurso) ou não, apenas computa o voto e essa análise é algo que fica a posteriori. Coluna importante caso queiramos fazer alguma análise dessas informações.

Essas informações foram obtidas [nessa lista](https://sig.tse.jus.br/ords/dwapr/r/seai/sig-candidaturas/) e foram processadas em um outro notebook.

In [24]:
path = 'C:\\Users\\ASUS\\Desktop\\Projetos\\Portfólio\\Projeto Urna\\PASTA AUXILIAR\\indeferidos_final.xlsx'

df_ind = pd.read_excel(path)

indeferidos = df_ind['NUMERO'].tolist()
COND = (df_eleicoes['NUMERO'].isin(indeferidos)) | ((df_eleicoes['NUMERO'] == 95) | (df_eleicoes['NUMERO'] == 96))


df_eleicoes['SITUACAO'] = np.where(COND, 0, 1)

In [25]:
COND2 = ((df_eleicoes['NUMERO'] == 95) | (df_eleicoes['NUMERO'] == 96))
df_check = df_eleicoes[COND2]

## Conclusão

E finalmente temos o nosso dataset pronto. 

Para fins de organização, vou utilizar o *dataset_eleicoes* **em outro notebook**. 

Particularmente foi mais fácil utilizar o Pandas por conta da quantidade de informações, tutoriais e vídeos internet afora explicando como utilizar cada comando. 

In [27]:
# Salvando o arquivo novo
df_eleicoes.to_excel("dataset_eleicoes.xlsx", index=False)